<a href="https://colab.research.google.com/github/aliakbarbadri/natural-language-inference/blob/master/snli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

https://www.tensorflow.org/datasets/catalog/snli


In [10]:
ds = tfds.load('snli')

In [24]:
train_ds = ds['train']
valid_ds = ds['validation']
test_ds = ds['test']
# 0: entailment, 1: neutral, 2: contradiction , -1:-

In [25]:
for example in train_ds.take(40):
  hypothesis, label, premise = example["hypothesis"], example["label"], example["premise"]
  if int(label) == 0:
    print(hypothesis)
    # print(label)
    print(premise)
    print("--------")

tf.Tensor(b'There are motorcycles in a lot.', shape=(), dtype=string)
tf.Tensor(b'A group of motorcycles and scooters are parked together in a lot in the middle of a street.', shape=(), dtype=string)
--------
tf.Tensor(b'A protester is pregnant', shape=(), dtype=string)
tf.Tensor(b'A pregnant protester holding up a pro-America sign.', shape=(), dtype=string)
--------
tf.Tensor(b'There are people with boxed displays.', shape=(), dtype=string)
tf.Tensor(b'Asian people sit with their boxed display of brown items.', shape=(), dtype=string)
--------
tf.Tensor(b'A boy and man are cooking meat.', shape=(), dtype=string)
tf.Tensor(b'At a street festival, a boy and a man cook some sort of "Texas Smoked" meat while pedestrians pass by.', shape=(), dtype=string)
--------
tf.Tensor(b'There are multiple people present.', shape=(), dtype=string)
tf.Tensor(b'A group of men in reflective gear are holding light sticks while standing on a wooden floor that has outdoor lighting.', shape=(), dtype=string)